# Player Search
This notebook explores the use of these two endpoints in retrieving individual NBA player statstics:
- https://v2.nba.api-sports.io/players
- https://v2.nba.api-sports.io/players/statistics

The code from this notebook will be used in V1 of the _Player Search_ feature.

In [1]:
from rel_bball_analytics import app

import pandas as pd
import requests

In [2]:
base_url = app.config["SPORTS_API_URL"]
headers = {"x-rapidapi-key": app.config["SPORTS_API_KEY"]}

### Get Player Info from Name

In [3]:
PLAYER_COLUMNS = [
    "id",
    "firstname",
    "lastname",
    "birth_date",
    "country",
    "height_feet",
    "height_inches",
    "weight_pounds",
    "jersey",
    "is_active",
    "position",
    "start_year",
    "pro_years",
    "college",
]

In [4]:
def get_raw_player_data(name):
    url = f"{base_url}players"
    params = {"name": name}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [5]:
def filter_nba_players(players):
    players["is_nba"] = players["leagues"].apply(lambda obj: "standard" in obj.keys())
    return players.loc[players.is_nba]

In [6]:
def clean_player_data(data):
    data["birth_date"] = data["birth"].apply(lambda obj: obj.get("date"))
    data["country"] = data["birth"].apply(lambda obj: obj.get("country"))
    data["start_year"] = data["nba"].apply(lambda obj: obj.get("start"))
    data["pro_years"] = data["nba"].apply(lambda obj: obj.get("pro"))
    data["height_feet"] = data["height"].apply(lambda obj: obj.get("feets"))
    data["height_inches"] = data["height"].apply(lambda obj: obj.get("inches"))
    data["weight_pounds"] = data["weight"].apply(lambda obj: obj.get("pounds"))
    data["jersey"] = data["leagues"].apply(lambda obj: obj["standard"].get("jersey"))
    data["is_active"] = data["leagues"].apply(lambda obj: obj["standard"].get("active"))
    data["position"] = data["leagues"].apply(lambda obj: obj["standard"].get("pos"))

    return data[PLAYER_COLUMNS]

In [7]:
player_data = get_raw_player_data(name="curry")
if player_data["results"] == 0:
    print("No player matches")

df_players = pd.DataFrame(player_data["response"])
df_players = filter_nba_players(players=df_players)
df_players = clean_player_data(data=df_players)

In [8]:
df_players

,id,firstname,lastname,birth_date,country,height_feet,height_inches,weight_pounds,jersey,is_active,position,start_year,pro_years,college
0,123,Seth,Curry,1990-08-23,USA,6,2,185,30,True,G,2013,7,Duke
1,124,Stephen,Curry,1988-03-14,USA,6,2,185,30,True,G,2009,12,Davidson


### Get Player Stats from ID

In [9]:
EXCLUDED_STATS_COLUMNS = [
    "player",
    "team",
    "game",
    "comment",
]

In [10]:
def get_raw_stats_data(player_id, season=2022):
    url = f"{base_url}players/statistics"
    params = {"id": player_id, "season": season}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [11]:
def time_to_int(time):
    if time is None or type(time) == int:
        return time
    
    if ":" in time:
        mins, secs = time.split(":")
        result = int(mins) + int(secs) / 60
        return round(result, 2)

    return round(int(time), 2)

In [12]:
def clean_stats_data(data):
    data["player_id"] = data["player"].apply(lambda obj: obj["id"])
    data["team_id"] = data["team"].apply(lambda obj: obj["id"])
    data["team_name"] = data["team"].apply(lambda obj: obj["name"])
    data["team_code"] = data["team"].apply(lambda obj: obj["code"])
    data["game_id"] = data["game"].apply(lambda obj: obj["id"])
    data["min"] = data["min"].apply(time_to_int)

    return data.drop(EXCLUDED_STATS_COLUMNS, axis=1)

In [13]:
stats_data = get_raw_stats_data(player_id=124, season=2022)
if stats_data["results"] == 0:
    print(f"No stats available")

df_stats = pd.DataFrame(stats_data["response"])
df_stats = df_stats.dropna(subset=["points", "min"])
df_stats = clean_stats_data(data=df_stats)

In [14]:
df_stats

,points,pos,min,fgm,fga,fgp,ftm,fta,ftp,tpm,...,pFouls,steals,turnovers,blocks,plusMinus,player_id,team_id,team_name,team_code,game_id
0,17,PG,16.97,7,13,53.8,0,0,0.0,3,...,1,0,4,1,-2,124,11,Golden State Warriors,GSW,10983
1,6,PG,13.08,1,7,14.3,3,3,100,1,...,0,0,4,0,-2,124,11,Golden State Warriors,GSW,10980
2,38,PG,37.00,12,20,60.0,6,6,100.0,8,...,0,0,4,1,+17,124,11,Golden State Warriors,GSW,11808
3,35,PG,39.00,13,21,61.9,5,5,100.0,4,...,1,2,3,0,+13,124,11,Golden State Warriors,GSW,11789
4,34,PG,35.00,10,19,52.6,10,11,90.9,4,...,3,2,3,0,-3,124,11,Golden State Warriors,GSW,11775
5,26,PG,38.00,7,16,43.8,8,8,100.0,4,...,2,1,5,0,-8,124,11,Golden State Warriors,GSW,11753
6,29,PG,43.00,9,25,36.0,5,5,100.0,6,...,1,3,3,2,+9,124,11,Golden State Warriors,GSW,11728
7,41,PG,38.00,12,28,42.9,11,12,91.7,6,...,1,0,4,0,+14,124,11,Golden State Warriors,GSW,11709
8,20,PG,34.00,8,15,53.3,1,2,50.0,3,...,2,0,8,1,-9,124,11,Golden State Warriors,GSW,11699
9,15,PG,23.00,6,12,50.0,1,1,100.0,2,...,3,0,0,1,+12,124,11,Golden State Warriors,GSW,11684


### Get Summary Stats (Per Game)

In [15]:
STATS_COLUMNS = {
    "points": "points",
    "min": "minutes_played",
    "fgm": "field_goals",
    "fga": "field_goal_attempts",
    "fgp": "field_goal_percentage",
    "tpm": "three_points_made",
    "tpa": "three_point_attempts",
    "tpp": "three_point_percentage",
    "ftm": "free_throws",
    "fta": "free_throw_attempts",
    "ftp": "free_throw_percentage",
    "offReb": "offensive_rebounds",
    "defReb": "defensive_rebounds",
    "totReb": "total_rebounds",
    "assists": "assists",
    "steals": "steals",
    "blocks": "blocks",
    "turnovers": "turnovers",
    "pFouls": "personal_fouls",
    "plusMinus": "plus_minus",
}

In [16]:
def get_summary_stats(data):
    stats_data = data[STATS_COLUMNS.keys()].rename(columns=STATS_COLUMNS)
    return stats_data.mean(axis=0, numeric_only=True) 

In [17]:
summary_stats = {
    "team": df_stats["team_code"].mode()[0],
    "position": df_stats["pos"].mode()[0],
    "games_played": len(df_stats),
    **get_summary_stats(data=df_stats)
}

In [18]:
summary_stats

{'team': 'GSW',
 'position': 'PG',
 'games_played': 41,
 'points': 28.4390243902439,
 'minutes_played': 33.44024390243902,
 'field_goals': 9.487804878048781,
 'field_goal_attempts': 19.24390243902439,
 'three_points_made': 4.658536585365853,
 'three_point_attempts': 11.048780487804878,
 'free_throws': 4.804878048780488,
 'free_throw_attempts': 5.195121951219512,
 'offensive_rebounds': 0.5365853658536586,
 'defensive_rebounds': 5.634146341463414,
 'total_rebounds': 6.170731707317073,
 'assists': 6.146341463414634,
 'steals': 0.9024390243902439,
 'blocks': 0.3902439024390244,
 'turnovers': 3.2439024390243905,
 'personal_fouls': 2.048780487804878}